In [1]:
# from Utills import TrainFX
import json
from requests.exceptions import Timeout
from datetime import date, datetime,timedelta


### Fetch Train info list

In [8]:
import requests
from bs4 import BeautifulSoup

def extract_train_numbers(html_file, trainlist):
    soup = BeautifulSoup(html_file, 'html.parser')

    train_numbers = []
    for tr in soup.find_all('tr'):
        if tr.find('td', text=True):
            train_number = tr.find('td', text=True).text
            trainlist.append(train_number)

    # return train_numbers

trainlist = []

# for i in range(1,6):
#     file = f'src{i}.html'
#     print(file)
#     with open(file, 'r') as f:
#         html = f.read()
#     extract_train_numbers(html, trainlist)
#     # trainlist.append(trainlist)

# print(len(trainlist))
# print(trainlist)


['14722', '09438', '05348', '22188', '17409', '07775', '07851', '08653', '08659', '08643', '08657', '08655', '08661', '08647', '08641', '08671', '08669', '08674', '08676', '08684', '08663', '08686', '08680', '08649', '20501', '15626', '05675', '05628', '07679', '12097', '02502', '12520', '01666', '07682', '07684', '07690', '07688', '13174', '07029', '15663', '05695', '12504', '06627', '06629', '12341', '12342', '04167', '04165', '04164', '14011', '20976', '13168', '01909', '04495', '12547', '11808', '01972', '11902', '04157', '12195', '22988', '01913', '05414', '05914', '12180', '15055', '14211', '22645', '22646', '22185', '22186', '03056', '09274', '04168', '04166', '09400', '19435', '19483', '19418', '15560', '09465', '19165', '20947', '20949', '19489', '19409', '12917', '12833', '19223', '22925', '01906', '19413', '19401', '22920', '20954', '12932', '82902', '12010', '09435', '09447', '19421', '09417', '22967', '12297', '12844', '20862', '11049', '19415', '20939', '09419', '19036', 

### Fetch Train Info

In [16]:
def getTrainInfo(train_number, timeout):
    unix_time = int(datetime.utcnow().timestamp()-1532)
    cookies = {
        '__gads' : 'ID=0b5cc381a6cc5d5e:T=1697635642:RT=1697635642:S=ALNI_MbSIbTJTVGP3rMCM8Xt4aDIub5Evg',
        '_gid' : 'GA1.3.1679543960.1697635644; _ga_SHTZYKNHG2=GS1.1.1697635639.1.1.1697635683.0.0.0',
        'JSESSIONID' : 'GA9C-MwGVJERPBL6MlhhbR39HY-Hq--Fog7TVrg5Lltr7FNiAraG!45373394; _ga=GA1.1.1336431025.1697635639',
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://www.irctc.co.in/nget/booking/check-train-schedule',
        'Content-Type': 'application/x-www-form-urlencoded',
        'greq': str(unix_time),
        'Content-Language': 'en',
        'Connection': 'keep-alive',
    }
    response = requests.get('https://www.irctc.co.in/eticketing/protected/mapps1/trnscheduleenquiry/'+train_number, headers=headers, timeout= timeout)
    return response
    
max_attempts = 3  # Maximum number of retry attempts
trainname = trainlist
train_info_list = []
faulty_train = []

for train in trainname:
    attempts = 0
    while attempts < max_attempts:
        try:
            response = getTrainInfo(train, timeout=2)  # Set an appropriate timeout value
            response.raise_for_status()  # Raise an HTTPError for bad responses
            train_info = json.loads(response.text)
            print("Train: ", train, " Info received")
            if 'errorMessage' not in train_info:
                train_info_list.append(train_info)
            else:
                faulty_train.append(train)
                print(f"Faulty train number {train}")
            break  # Break out of the retry loop if successful
        except Timeout as e:
            print(f"Timeout error for train {train}. Retrying... ({attempts + 1}/{max_attempts})")
            attempts += 1
        except requests.RequestException as e:
            print(f"Request failed for train {train} with error: {e}")
            break  # Break out of the loop for other request errors

    if attempts == max_attempts:
        print(f"Error: Max attempts reached for train {train}")

# store the train_info_list in a new file
with open('train_info.json', 'w') as f:
    json.dump(train_info_list, f)

Train:  12723  Info received
Train:  22416  Info received
Faulty train number 22416
Train:  12724  Info received
Train:  12707  Info received
Train:  15609  Info received
Faulty train number 15609
Train:  15909  Info received
Train:  18242  Info received
Train:  11266  Info received
Train:  54703  Info received
Faulty train number 54703
Train:  07509  Info received
Train:  09427  Info received
Faulty train number 09427
Train:  09018  Info received
Train:  14804  Info received
Train:  04804  Info received
Faulty train number 04804
Train:  09421  Info received
Faulty train number 09421
Train:  19130  Info received
Faulty train number 19130
Train:  02713  Info received
Faulty train number 02713
Train:  19420  Info received
Faulty train number 19420
Train:  09417  Info received
Train:  12931  Info received
Train:  12267  Info received
Train:  12298  Info received
Train:  09425  Info received
Faulty train number 09425
Train:  19223  Info received
Train:  09413  Info received
Faulty train nu

In [18]:
print(len(trainlist)-len(faulty_train))

1834
